In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName("pyspark project").getOrCreate()

In [17]:
walmartdf = spark.read.format("jdbc") \
 .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
 .option("driver","com.mysql.jdbc.Driver") \
 .option("dbtable", "walmartdata") \
 .option("user", "root").option("password", "cloudera") \
 .load()

In [38]:
walmartdf.show(3)

+-------------------+-----+-----+-----+-----+--------+------------------+
|               Date| Open| High|  Low|Close|  Volume|         Adj_Close|
+-------------------+-----+-----+-----+-----+--------+------------------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|         52.078475|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|         51.825539|
+-------------------+-----+-----+-----+-----+--------+------------------+
only showing top 3 rows



In [20]:
walmartdf.createOrReplaceTempView("walmartdata")

### 1.Create a new dataframe with a column called VH Ratio that is the ratio of the High Price versus  volume of stock traded for a day.

In [22]:
VHratio=spark.sql("select date, round(Volume/High,1) as VHratio from walmartdata")

In [23]:
VHratio.show(3)

+-------------------+--------+
|               date| VHratio|
+-------------------+--------+
|2012-01-03 00:00:00|207481.2|
|2012-01-04 00:00:00|158961.1|
|2012-01-05 00:00:00|214159.7|
+-------------------+--------+
only showing top 3 rows



### Exporting output to mysql

In [ ]:
HVratio.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol1") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()

## 2.What day had the Peak High in Price?

In [24]:
PeakHigh=spark.sql("select Date, High as Peak_High_price from walmartdata order by Peak_High_price desc limit 10")

In [25]:
PeakHigh.show()

+-------------------+---------------+
|               Date|Peak_High_price|
+-------------------+---------------+
|2015-01-13 00:00:00|          90.97|
|2015-01-08 00:00:00|          90.67|
|2015-01-09 00:00:00|          90.39|
|2015-01-12 00:00:00|          90.31|
|2015-01-23 00:00:00|          89.26|
|2015-01-26 00:00:00|          89.16|
|2015-01-07 00:00:00|          88.68|
|2015-01-14 00:00:00|          88.52|
|2015-01-27 00:00:00|          88.46|
|2015-01-22 00:00:00|           88.4|
+-------------------+---------------+



In [ ]:
PeakHigh.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol2") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()

## 3.What is the max and min of the Volume column?

In [26]:
MaxMinVolume=spark.sql("select max(volume) as MaxVolume, min(volume) as MinVolume from walmartdata")

In [27]:
MaxMinVolume.show()

+---------+---------+
|MaxVolume|MinVolume|
+---------+---------+
| 80898100|  2094900|
+---------+---------+



In [57]:
MaxMinVolume.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol3") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()

## 4.How many days was the Close lower than 60 dollars?

In [28]:
CloseLower=spark.sql("select date, close from walmartdata where close <=60")

In [39]:
CloseLower.show(5)

+-------------------+-----+
|               date|close|
+-------------------+-----+
|2012-01-04 00:00:00|59.71|
|2012-01-05 00:00:00|59.42|
|2012-01-06 00:00:00| 59.0|
|2012-01-09 00:00:00|59.18|
|2012-01-10 00:00:00|59.04|
+-------------------+-----+
only showing top 5 rows



In [ ]:
CloseLower.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol4") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()

### 5.What percentage of the time was the High greater than 80 dollars ?
### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [63]:
High_greater=spark.sql("SELECT round(sum(if(high>80,1,0))*100/count(*),2) AS percentage FROM walmartdata")

In [64]:
High_greater.show()

+----------+
|percentage|
+----------+
|      9.14|
+----------+



In [66]:
solution5=High_greater.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol5") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()

### 6.What is the Pearson correlation between High and Volume?

In [49]:
pearson_correlation = spark.sql("SELECT corr(high, volume) AS pearson_correlation FROM walmartdata")

In [51]:
pearson_correlation.show()

+--------------------+
| pearson_correlation|
+--------------------+
|-0.33843260582148915|
+--------------------+



In [ ]:
pearson_correlation.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol6") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()

### 7.What is the max High per year?

In [34]:
max_high=spark.sql("select year(date) as year, max(high) as max_high_year from walmartdata group by year")
max_high.show()

+----+-------------+
|year|max_high_year|
+----+-------------+
|2015|        90.97|
|2013|        81.37|
|2014|        88.09|
|2012|         77.6|
|2016|        75.19|
+----+-------------+



In [ ]:
max_high.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol7") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()

### 8.What is the average Close for each Calendar Month?

In [36]:
average_close=spark.sql("select month(date) as month, avg(close) as average_close from walmartdata group by month order by month")

In [37]:
average_close.show()

+-----+-----------------+
|month|    average_close|
+-----+-----------------+
|    1|71.44801980198022|
|    2|71.30680412371134|
|    3|71.77794392523363|
|    4|72.97361904761907|
|    5|72.30971698113206|
|    6|72.49537735849057|
|    7|74.43971962616824|
|    8|73.02981818181819|
|    9|72.18411764705883|
|   10|71.57854545454546|
|   11|72.11108910891085|
|   12|72.84792452830189|
+-----+-----------------+



In [ ]:
average_close.write.format("jdbc") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("url", "jdbc:mysql://localhost:3306/sparkproject") \
                    .option("dbtable","sol8") \
                    .option("user", "root").option("password", "cloudera") \
                    .save()